In [38]:
import pandas as pd
import os
import seaborn as sns
import numpy as np
import nltk     
import re
import spacy
import string
from nltk.corpus import wordnet
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
import graphviz 
from sklearn import tree
from sklearn import metrics
from wordcloud import WordCloud
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
DATA_DIR = os.path.join( '..', 'data')
RAW_DATA_DIR = os.path.join(DATA_DIR, 'raw')
PROCESSED_DATA_DIR = os.path.join(DATA_DIR, 'processed')

In [39]:
file_path=os.path.join(PROCESSED_DATA_DIR,"sms.csv")
sms=pd.read_csv(file_path)
sms.head(2)




,Label,SMS,lenght,Tokens,lemmatized_text,SMS_without_stopwords
0,0,"gone until jurong point, crazy.. available onl...",111,"['gone', 'until', 'jurong', 'point', ',', 'cra...","['go', 'until', 'jurong', 'point', ',', 'crazy...","['go', 'jurong', 'point', ',', 'crazy', '..', ..."
1,0,ok lar... joking wif you oni...,29,"['ok', 'lar', '...', 'joking', 'wif', 'you', '...","['ok', 'lar', '...', 'joke', 'wif', 'you', 'on...","['ok', 'lar', '...', 'joke', 'wif', 'oni', '...']"


In [40]:
#Para entrenar un modelo de regresión logística, necesitas convertir tus datos de texto en
#características numéricas. Puedes hacer esto utilizando un enfoque de "bag of words"
#(bolsa de palabras) con CountVectorizer:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sms['SMS'])
tamanio_X = X.size
X_new=['get','go','let','come','want','ok','good','know','call','like','text','free','mobile','win','stop','claim','reply','message','prize','tone']
list_key=list(range(20))
new_columns = list_key[0:20]
X_new=X[new_columns]

In [41]:
# Supongamos que ya tienes 'sms1' y 'list_key' definidos
# También, supongo que 'sms1' es un DataFrame de Pandas.

# Crear el vectorizador y ajustar/transformar los datos
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sms['SMS'])

# Obtener las columnas correspondientes a las palabras en 'list_key'
#list_key = ["want","get"]
list_key = X_new=['get','go','let','come','want','ok','good','know','call','like','text','free','mobile','win','stop','claim','reply','message','prize','tone']
selected_columns = [vectorizer.vocabulary_[word] for word in list_key if word in vectorizer.vocabulary_]

# Seleccionar solo las columnas relevantes
X_new = X[:, selected_columns]

# Convertir X_new a un DataFrame de Pandas
X_new_df = pd.DataFrame(X_new.toarray(), columns=['X_' + str(i) for i in selected_columns])

# Concatenar X_new_df con el conjunto de datos original sms1
sms = pd.concat([sms, X_new_df], axis=1)

# Verificar las dimensiones de X_new
print(X_new.shape)

print(sms.shape)
sms.head(2)

(5263, 19)
(5263, 25)


,Label,SMS,lenght,Tokens,lemmatized_text,SMS_without_stopwords,X_3544,X_4641,X_2177,X_8366,...,X_7703,X_3390,X_5148,X_8533,X_7365,X_2081,X_6480,X_5049,X_6151,X_7899
0,0,"gone until jurong point, crazy.. available onl...",111,"['gone', 'until', 'jurong', 'point', ',', 'cra...","['go', 'until', 'jurong', 'point', ',', 'crazy...","['go', 'jurong', 'point', ',', 'crazy', '..', ...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,ok lar... joking wif you oni...,29,"['ok', 'lar', '...', 'joking', 'wif', 'you', '...","['ok', 'lar', '...', 'joke', 'wif', 'you', 'on...","['ok', 'lar', '...', 'joke', 'wif', 'oni', '...']",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
#check all the columns
sms.columns


Index(['Label', 'SMS', 'lenght', 'Tokens', 'lemmatized_text',
       'SMS_without_stopwords', 'X_3544', 'X_4641', 'X_2177', 'X_8366',
       'X_5574', 'X_3619', 'X_4498', 'X_1841', 'X_4670', 'X_7703', 'X_3390',
       'X_5148', 'X_8533', 'X_7365', 'X_2081', 'X_6480', 'X_5049', 'X_6151',
       'X_7899'],
      dtype='object')

In [43]:
sms_final =sms[['Label', 'X_3544', 'X_4641', 'X_2177',
       'X_8366', 'X_5574', 'X_3619', 'X_4498', 'X_1841', 'X_4670', 'X_7703',
       'X_3390', 'X_5148', 'X_8533', 'X_7365', 'X_2081', 'X_6480', 'X_5049',
       'X_6151', 'X_7899']]

In [44]:
#sms_final separated in train validation and test
sms_train, sms_test = train_test_split(sms_final, test_size=0.2, random_state=42)

sms_train, sms_val = train_test_split(sms_train, test_size=0.2, random_state=42)


In [45]:
#save in csv the train, validation and test in the folder processed
sms_train.to_csv(os.path.join(PROCESSED_DATA_DIR,"sms_train.csv"),index=False)
sms_val.to_csv(os.path.join(PROCESSED_DATA_DIR,"sms_val.csv"),index=False)
sms_test.to_csv(os.path.join(PROCESSED_DATA_DIR,"sms_test.csv"),index=False)
